In [ ]:
# AI and ML - group project

# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Importing dataset
df = pd.read_csv(r'C:/Users/silva/Documents/Faculdade/Intercâmbio/AI and Machine Learning/Group project/customer_segmentation.csv')
print(df)

# Fixing the date
datelist= ['order_purchase_timestamp','order_approved_at','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date','shipping_limit_date']

# Converting the list of date variables to a pandas datetime object
for c in datelist:
    df[c]=pd.to_datetime(df[c])

# Separating the date and time in the order column they might be interesting variables for the analysis
df['order_date'] = [d.date() for d in df['order_purchase_timestamp']]
df['order_time'] = [d.time() for d in df['order_purchase_timestamp']]

# Taking a look at the data structure
df.head()
df.info()
df.describe()
df.shape

# Data preparation

# Removing duplicates and resetting the index of the dataframe
df.duplicated().sum()
df.drop_duplicates(keep = 'first', inplace = True)
df.reset_index(drop = True, inplace = True)

# Checking for missing values
df.isnull().sum()

# Checking for outliers
plt.figure()
df.reset_index().plot(kind='scatter', x='index', y='payment_installments')
plt.figure()
df.reset_index().plot(kind='scatter', x='index', y='payment_value')

# Checking the mean and standar deviation of the payment value (to check how far is the outlier observed in the scatterplot) 
mean_pv = df["payment_value"].mean() # mean of 195.2
std_pv = df["payment_value"].std() # std of 295.5

# Seeing how many outliers there are, the output prints 5 payments higher than 4000
# Value of 4000 decided by looking at the scatterplot
outlier1 = df[df['payment_value'] > 4000]
print('\nOutlier dataframe:\n', outlier1)

# Deleting 5 outliers out of 13718 other orders and resetting the index again
df = df.drop(df[df.payment_value > 4000].index)
df = df.reset_index(drop = True)

# Continuing to look at other variables by plotting scatterplots
plt.figure()
df.reset_index().plot(kind='scatter', x='index', y='payment_value') # re-plotting after removing outliers
plt.figure()
df.reset_index().plot(kind='scatter', x='index', y='price')
plt.figure()
df.reset_index().plot(kind='scatter', x='index', y='freight_value')
plt.figure()

# Creating a correlation matrix heatmap to look at the relation between variables
sns.heatmap(df.corr(), annot=True)
plt.figure()

# Counting customers per city and per state and creating dataframes with the purpose of plotting a graph
customerstate = df.groupby('customer_state').count()['customer_id'].reset_index()
customercity= df.groupby('customer_city').count()['customer_id'].reset_index()

# Graph of numer of customers in each state
plt.figure(figsize = (25,7))
plt.subplot(121)
sns.barplot(data = customerstate.sort_values('customer_id', ascending = False), x = 'customer_state', y = 'customer_id')
plt.title('Number of Customers in a State')
plt.xlabel('State')
plt.ylabel('Number of Customers')

# Graph of top 10 cities with most customers
plt.subplot(122)
sns.barplot(data = customercity.sort_values('customer_id', ascending = False).nlargest(10,'customer_id'), x = 'customer_id', y = 'customer_city')
plt.title('Cities with more Customers')
plt.xlabel('City')
plt.ylabel('Number of Customers');

# Grouping similar categories of product category names in case we use this variable in future analysis (encoding becomes easier, there would be less dummies)
df['product_category_name_english'] = df['product_category_name_english'].replace(['art', 'arts_and_craftmanship', 'sports_leisure',
                                                                                   'garden_tools', 'flowers', 'music', 'musical_instruments',
                                                                                   'books_general_interest', 'books_imported', 'books_technical'], 'hobbies')
df['product_category_name_english'] = df['product_category_name_english'].replace(['air_conditioning', 'bed_bath_table', 'furniture_bedroom',
                                                                                   'furniture_decor', 'furniture_living_room', 'home_appliances',
                                                                                   'home_appliances_2', 'home_comfort_2', 'home_confort',
                                                                                   'home_construction', 'housewares', 'kitchen_dining_laundry_garden_furniture',
                                                                                   'small_appliances', 'small_appliances_home_oven_and_coffee', 'office_furniture',
                                                                                   'signaling_and_security', 'stationery', 'luggage_accessories'], 'home_products')
df['product_category_name_english'] = df['product_category_name_english'].replace(['drinks', 'food', 'food_drink'], 'food_drink')
df['product_category_name_english'] = df['product_category_name_english'].replace(['construction_tools_construction', 'construction_tools_lights',
                                                                                   'construction_tools_safety', 'costruction_tools_garden', 
                                                                                   'costruction_tools_tools'], 'construction_tools')
df['product_category_name_english'] = df['product_category_name_english'].replace(['audio', 'auto', 'cds_dvds_musicals', 'cine_photo', 'computers', 
                                                                                   'computers_accessories', 'consoles_games', 'dvds_blu_ray', 'electronics',
                                                                                   'fixed_telephony', 'telephony', 'tablets_printing_image'], 'electronic_gadgets')
df['product_category_name_english'] = df['product_category_name_english'].replace(['fashion_bags_accessories', 'fashion_childrens_clothes', 'fashion_male_clothing',
                                                                                   'fashion_shoes', 'fashion_underwear_beach', 'health_beauty', 'perfumery'], 'fashion_beauty')
df['product_category_name_english'] = df['product_category_name_english'].replace(['baby', 'diapers_and_hygiene', 'toys', 'party_supplies', 'pet_shop',
                                                                                   'christmas_supplies', 'cool_stuff', 'watches_gifts'], 'family_festivities')
df['product_category_name_english'] = df['product_category_name_english'].replace(['industry_commerce_and_business', 'market_place'], 'sellers')


# Counting the product categories to check which one is most and less ordered
best_seller_p = df['product_category_name_english'].value_counts().reset_index().nlargest(5,'product_category_name_english')
worst_seller_p = df['product_category_name_english'].value_counts().reset_index().nsmallest(5,'product_category_name_english')

# Graphs aesthetic
plt.figure(figsize = (15,12))
green_color = sns.color_palette()[3]
red_color = sns.color_palette()[2]

# Graph of top 10 most ordered products
plt.subplot(211)
sns.barplot(data = best_seller_p, x = 'product_category_name_english', y = 'index', color = green_color)
plt.title('Top 5 Product Categories Ordered')
plt.xlabel('Number of Orders')
plt.ylabel('Product Category');

# Graph of top 10 less ordered products
plt.subplot(212)
sns.barplot(data = worst_seller_p, x = 'product_category_name_english', y = 'index', color =red_color )
plt.title('Lowest 5 Product Categories Ordered')
plt.xlabel('Number of Orders')
plt.ylabel('Product Category');

# Counting different payment types to plot which ones are more used
payments_types = df['payment_type'].value_counts().reset_index()

plt.figure(figsize = (25,7))

# Graph of number of orders per payment type
plt.subplot(121)
sns.barplot(data = payments_types, x = 'index', y = 'payment_type')
plt.title('Orders by Payment type')
plt.xlabel('Payment Type')
plt.ylabel('Number of Orders');

# Graph of number of orders per number of payment installments
plt.subplot(122)
sns.barplot(data = df['payment_installments'].value_counts().reset_index(), x = 'index', y = 'payment_installments')
plt.title('Count of Orders With Number of Payment Installments')
plt.xlabel('Number of Payment Installments')
plt.ylabel('Count of Orders');

# Encoding variables 
df = pd.get_dummies(df,prefix=['customer_state: '], columns = ['customer_state'], drop_first=True)
df = pd.get_dummies(df,prefix=['payment_type: '], columns = ['payment_type'], drop_first=True)
df = pd.get_dummies(df,prefix=['seller_state: '], columns = ['seller_state'], drop_first=True)
df = pd.get_dummies(df,prefix=['product_category_name: '], columns = ['product_category_name_english'], drop_first=True)

# Calculating the RFM

# Recency = time since a customer's last purchase
# Calculating each purchasing time stamp minus the most recent purchase timestamp (max)
df['recency'] = max(df['order_purchase_timestamp'])-df['order_purchase_timestamp']
# Getting the minimum recency value for each customer (customer with multiple purchases ended up with multiple recencies, therefore calculating the most recent purchase)
df['recency'] = df.groupby(['customer_unique_id'], as_index=False)['recency'].transform('min')
# Keeping only the days in the column, leaving out the time
df['recency'] = df['recency'].dt.days

# Frequency = how many times has a customer made a purchase
# Counting purchases per customer unique id and adding values to a new column in the dataset
df['frequency'] = df.groupby(['customer_unique_id'], as_index=False)['order_id'].transform('count')

# Monetary = total amount a customer has spend purchasing products
# Calculating it by summing all the payment values a customer has spent
df['monetary'] = df.groupby(['customer_unique_id'], as_index=False)['payment_value'].transform('sum')

# Creating an empty column to calculate recency score
df['recency_score'] = ''

# For function that inputs the score in the empty column based on the conditions set
# The numbers were chosen based of the recency mean, std, max and min + our reasoning considered aspects like "how many months could we consider the customer inactive?"
print('recency mean: ', df['recency'].mean())
print('recency std: ', df['recency'].std())
print('recency max: ', df['recency'].max())
print('recency min: ', df['recency'].min())

for i in df.index:
    if df['recency'][i] <= 30:
        df['recency_score'][i] = 5
    elif (df['recency'][i] > 30) and (df['recency'][i] <= 60):
        df['recency_score'][i] = 4
    elif (df['recency'][i] > 60) and (df['recency'][i] <= 120):
        df['recency_score'][i] = 3
    elif (df['recency'][i] > 120) and (df['recency'][i] <= 180):
        df['recency_score'][i] = 2
    elif (df['recency'][i] > 180):
        df['recency_score'][i] = 1

# Creating an empty column to calculate frequency score
df['frequency_score'] = ''

# For function that inputs the score in the empty column based on the conditions set
# The numbers were chosen based of the recency mean, std, max and min + our reasoning considered aspects like "most customers purchased only one time, and max purchases were 13, how can we score from 0 to above"
print('frequency mean: ', df['frequency'].mean())
print('frequency std: ', df['frequency'].std())
print('frequency max: ', df['frequency'].max())
print('frequency min: ', df['frequency'].min())

for i in df.index:
    if df['frequency'][i] >= 10:
        df['frequency_score'][i] = 5
    elif (df['frequency'][i] >= 6) and (df['frequency'][i] < 10):
        df['frequency_score'][i] = 4
    elif (df['frequency'][i] >= 4) and (df['frequency'][i] < 6):
        df['frequency_score'][i] = 3
    elif (df['frequency'][i] >= 2) and (df['frequency'][i] < 4):
        df['frequency_score'][i] = 2
    elif (df['frequency'][i] == 1):
        df['frequency_score'][i] = 1

# Creating an empty column to calculate monetary score        
df['monetary_score'] = ''

# For function that inputs the score in the empty column based on the conditions set
# The numbers were chosen based of the recency mean, std, max and min + our reasoning considered aspects like "the mean of the payment was 395.1, and the std is 1090.5, how can we score the customers based on spending"
print('monetary mean: ', df['monetary'].mean())
print('monetary std: ', df['monetary'].std())
print('monetary max: ', df['monetary'].max())
print('monetary min: ', df['monetary'].min())

for i in df.index:
    if df['monetary'][i] > 500:
        df['monetary_score'][i] = 5
    elif (df['monetary'][i] > 250) and (df['monetary'][i] <= 500):
        df['monetary_score'][i] = 4
    elif (df['monetary'][i] > 150) and (df['monetary'][i] <= 250):
        df['monetary_score'][i] = 3
    elif (df['monetary'][i] > 100) and (df['monetary'][i] <= 150):
        df['monetary_score'][i] = 2
    elif (df['monetary'][i] <= 100):
        df['monetary_score'][i] = 1
        
# Uniting the scores in the same columns to find segments such as 555 (5 score for all rfm)
df['rfm_segment'] = df['recency_score'].astype(str) + df['frequency_score'].astype(str) + df['monetary_score'].astype(str)

# Turning all numbers from last created columns to numeric variables to use in future analysis
df['rfm_segment'] = pd.to_numeric(df['rfm_segment'])
df['recency_score'] = pd.to_numeric(df['recency_score'])
df['frequency_score'] = pd.to_numeric(df['frequency_score'])
df['monetary_score'] = pd.to_numeric(df['monetary_score'])

# Creating a dataframe with the rfm variables that are going to be used in the clustering algorithms
rfm = df[['recency','frequency','monetary' ]]

# Standardizing the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
rfm_std = scaler.fit_transform(rfm)

#-------------------------------------------------------------------------------------


# K means
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score 
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score


import plotly as py
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings


from itertools import product


# Using eblow method to find out optimal k value with WCSS score

WCSS = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(rfm_std)
    WCSS.append(kmeans.inertia_)
plt.figure()    
plt.plot(range(1, 11), WCSS, marker='o', label='line with marker')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()



# Fitting K-Means to the dataset
# Use fit_predict to cluster the dataset
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(rfm_std)

# Visualising the clusters
plt.scatter(rfm_std[y_kmeans == 0, 0], rfm_std[y_kmeans == 0, 1], s = 100, c = 'red', label = 'Cluster 1')
plt.scatter(rfm_std[y_kmeans == 1, 0], rfm_std[y_kmeans == 1, 1], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(rfm_std[y_kmeans == 2, 0], rfm_std[y_kmeans == 2, 1], s = 100, c = 'green', label = 'Cluster 3')

plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'yellow', label = 'Centroids')
plt.title('Clusters of customers')
plt.legend()
plt.show()


# Calculate cluster validation metrics
score_kemans_s = silhouette_score(rfm_std,kmeans.labels_, metric='euclidean')
score_kemans_c = calinski_harabasz_score(rfm_std, kmeans.labels_)
score_kemans_d = davies_bouldin_score(rfm_std, y_kmeans)
print('Silhouette Score: %.3f' % score_kemans_s)
print('Calinski Harabasz Score: %.3f' % score_kemans_c)
print('Davies Bouldin Score: %.3f' % score_kemans_d)






# hierarchical clustering
#Dendrogram for Hierarchical Clustering
#♥Using the dendrogram to find the optimal number of clusters

import scipy.cluster.hierarchy as sch
dendrogram = sch.dendrogram(sch.linkage(rfm_std, method = 'ward'))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()

# Fitting Hierarchical Clustering to the dataset
from sklearn.cluster import AgglomerativeClustering
hc = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'ward')
y_hc = hc.fit_predict(rfm_std)



# Agglomerative clustering
from numpy import unique
from numpy import where
from sklearn.cluster import AgglomerativeClustering

# define the model

# retrieve unique clusters
clusters = unique(y_hc)



# Calculate cluster validation metrics
score_AGclustering_s = silhouette_score(rfm_std, hc.labels_ , metric='euclidean')
score_AGclustering_c = calinski_harabasz_score(rfm_std, hc.labels_ ,)
score_AGclustering_d = davies_bouldin_score(rfm_std, y_hc)
print('Silhouette Score: %.3f' % score_AGclustering_s)
print('Calinski Harabasz Score: %.3f' % score_AGclustering_c)
print('Davies Bouldin Score: %.3f' % score_AGclustering_d)



# Visualising the clusters
#plt.scatter(rfm_std[y_hc == 0, 0], rfm_std[y_hc == 0, 1], s = 100, c = 'red', label = 'Cluster 1')
#plt.scatter(rfm_std[y_hc == 1, 0], rfm_std[y_hc == 1, 1], s = 100, c = 'blue', label = 'Cluster 2')
#plt.scatter(rfm_std[y_hc == 2, 0], rfm_std[y_hc == 2, 1], s = 100, c = 'green', label = 'Cluster 3')
#plt.scatter(rfm_std[y_hc == 3, 0], rfm_std[y_hc == 3, 1], s = 100, c = 'cyan', label = 'Cluster 4')
#plt.scatter(rfm_std[y_hc == 4, 0], rfm_std[y_hc == 4, 1], s = 100, c = 'magenta', label = 'Cluster 5')
#plt.scatter(X[y_hc == 5, 0], X[y_hc == 5, 1], s = 100, c = 'black', label = 'Cluster 6')
#plt.scatter(X[y_hc == 6, 0], X[y_hc == 6, 1], s = 100, c = 'orange', label = 'Cluster 7')
#plt.title('Clusters of customers')
##plt.legend()
#plt.show()


















#DBSCAN METHOD 
#Unlike k-means, DBSCAN will figure out the number of clusters.
#DBSCAN works by determining whether the minimum number of points are close enough
#to one another to be considered part of a single cluster. 
#DBSCAN is very sensitive to scale since epsilon is a fixed value for the maximum distance between two points.

# we find a suitable value for epsilon by calculating the distance to the nearest n points for each point,
# sorting and plotting the results. Then we look to see where the change is most pronounced
# (think of the angle between your arm and forearm) and select that as epsilon.
#We can calculate the distance from each point to its closest neighbour using the NearestNeighbors.
# The point itself is included in n_neighbors. The kneighbors method returns two arrays, 
#one which contains the distance to the closest n_neighbors points and the other which contains the index for each of those points.

rfm_std_df = pd.DataFrame(rfm_std, columns = ['recency','frequency','monetary'])



#tuning the paramiters 


min_sample = 2*len(rfm_std_df.columns)
print(min_sample) #equale to 6

import numpy as np
from sklearn.neighbors import NearestNeighbors
# n_neighbors = 5 as kneighbors function returns distance of point to itself (i.e. first column will be zeros) 
nbrs = NearestNeighbors(n_neighbors=7).fit(rfm_std_df)
# Find the k-neighbors of a point
neigh_dist, neigh_ind = nbrs.kneighbors(rfm_std_df)
# sort the neighbor distances (lengths to points) in ascending order
# axis = 0 represents sort along first axis i.e. sort along row
sort_neigh_dist = np.sort(neigh_dist, axis=0)
import matplotlib.pyplot as plt
k_dist = sort_neigh_dist[:, 6]
plt.plot(k_dist)
plt.axhline(y=0.4, linewidth=1, linestyle='dashed', color='k')
plt.ylabel("k-NN distance")
plt.xlabel("Sorted observations (4th NN)")
plt.show()


from sklearn.cluster import DBSCAN
dbscan_model = DBSCAN(eps=0.4 , min_samples=6).fit(rfm_std_df)



#fitting the model 



labels = dbscan_model.labels_
print(labels)
rfm_std_df["LABEL"] = labels
rfm_std_df.head(50)

#sns.pairplot( rfm_std_df, hue='LABEL', vars=['recency', 'frequency','monetary'] )
#plt.show()


g = sns.PairGrid(rfm_std_df , hue = 'LABEL', palette="Paired")
g=g.add_legend()
g = g.map(sns.scatterplot)
print(g)



# fit model and predict clusters
yhat = dbscan_model.fit_predict(rfm_std_df)
# retrieve unique clusters
clusters = unique(yhat)
print(clusters)
# Calculate cluster validation metrics
score_dbsacn_s = silhouette_score(rfm_std_df, yhat, metric='euclidean')
score_dbsacn_c = calinski_harabasz_score(rfm_std_df, yhat)
score_dbsacn_d = davies_bouldin_score(rfm_std_df,yhat )
print('Silhouette Score: %.3f' % score_dbsacn_s)
print('Calinski Harabasz Score: %.3f' % score_dbsacn_c)
print('Davies Bouldin Score: %.3f' % score_dbsacn_d)
































